<a href="https://colab.research.google.com/github/jamessutton600613-png/GC/blob/main/Copy_of_Untitled260.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os, json, hashlib
import numpy as np
import gemmi
from pyscf import gto, dft
from pyscf.dft import numint

project_root = "/content/gqr_fe4s4_ispG"
os.makedirs(project_root, exist_ok=True)
os.chdir(project_root)
print("Working in:", os.getcwd())

def sha512_of_file(path):
    h = hashlib.sha512()
    with open(path, "rb") as f:
        for chunk in iter(lambda: f.read(8192), b""):
            h.update(chunk)
    return h.hexdigest()

def load_fes_from_cif(cif_path):
    doc = gemmi.cif.read_file(cif_path)
    block = doc.sole_block()
    st = gemmi.make_structure_from_block(block)
    model = st[0]

    coords, syms = [], []
    for ch in model:
        for res in ch:
            for at in res:
                el = at.element.name.upper()
                if el.startswith("FE") or el == "S":
                    syms.append("Fe" if el.startswith("FE") else "S")
                    coords.append([at.pos.x, at.pos.y, at.pos.z])

    return np.array(coords), np.array(syms)

def select_fe4s4_centroid(coords, syms, n_s=4):
    """Pick 4 Fe + n_s nearest S to Fe centroid (no connectivity check)."""
    fe_idx = np.where(syms == "Fe")[0]
    s_idx  = np.where(syms == "S")[0]
    if fe_idx.size < 4 or s_idx.size < n_s:
        raise RuntimeError("Not enough Fe/S atoms.")

    fe_centroid = coords[fe_idx].mean(axis=0)
    dists = np.linalg.norm(coords[s_idx] - fe_centroid, axis=1)
    order = np.argsort(dists)
    s_sel = s_idx[order[:n_s]]

    sel = np.concatenate([fe_idx, s_sel])
    print("  Selected atom indices:", sel)
    return sel

def curvature_light(symbols, coords, box=3.0, ngrid=24):
    atom_str = "\n".join(f"{s} {x} {y} {z}"
                         for s,(x,y,z) in zip(symbols, coords))
    mol = gto.M(atom=atom_str, basis="sto-3g", spin=0, charge=0)
    mf = dft.RKS(mol); mf.xc = "PBE"; mf.max_cycle = 80; mf.conv_tol = 1e-5
    mf.kernel()

    xs = np.linspace(-box, box, ngrid)
    X, Y, Z = np.meshgrid(xs, xs, xs, indexing="ij")
    grid = np.stack([X.ravel(), Y.ravel(), Z.ravel()], axis=1)

    ni = numint.NumInt()
    ao = ni.eval_ao(mol, grid)
    rho = ni.eval_rho(mol, ao, mf.make_rdm1()).reshape((ngrid,ngrid,ngrid))

    dx = xs[1] - xs[0]
    lap = (np.gradient(np.gradient(rho,dx,axis=0),dx,axis=0) +
           np.gradient(np.gradient(rho,dx,axis=1),dx,axis=1) +
           np.gradient(np.gradient(rho,dx,axis=2),dx,axis=2))
    mask = rho > 1e-3
    kappa = np.abs(lap[mask].ravel())
    return np.clip(kappa, 0, np.percentile(kappa, 99.5))

def modeB(coords, symbols, amp=0.20):
    C = coords.copy()
    Fe = np.where(symbols == "Fe")[0]
    S  = np.where(symbols == "S")[0]

    pairs = []
    for fi in Fe:
        r = C[fi]
        d = [np.linalg.norm(C[si] - r) for si in S]
        si = S[int(np.argmin(d))]
        pairs.append((fi, si))

    def stretch(C, fi, si, d):
        v = C[si] - C[fi]; L = np.linalg.norm(v)
        if L < 1e-6: return C
        u = v/L
        C[fi] -= 0.5*d*u
        C[si] += 0.5*d*u
        return C

    if len(pairs) >= 2:
        stretch(C, pairs[0][0], pairs[0][1], +amp)
        stretch(C, pairs[1][0], pairs[1][1], -amp)
    return C

L_EFF = 3.0
BINS  = list(range(40, 126))

def delta_teeth(kappa_base, kappa_mode):
    rows = []
    for b in BINS:
        cb, _ = np.histogram(kappa_base, b, density=True)
        cm, _ = np.histogram(kappa_mode, b, density=True)
        tb = int(np.sum(cb < 1e-9))
        tm = int(np.sum(cm < 1e-9))
        rows.append({
            "bins": b,
            "Delta_r_A": L_EFF / b,
            "Delta_teeth": tm - tb
        })
    return rows

for tag, cif in [("4S3D_stageD", "4S3D.cif"),
                 ("4S3F_stageF", "4S3F.cif")]:
    print(f"\n=== PATCH {tag} from {cif} ===")
    coords, syms = load_fes_from_cif(cif)
    sel = select_fe4s4_centroid(coords, syms, n_s=4)
    sub_syms   = syms[sel]
    sub_coords = coords[sel] - coords[sel].mean(axis=0)

    k_base = curvature_light(sub_syms, sub_coords)
    coords_mode = modeB(sub_coords, sub_syms, amp=0.20)
    k_mode = curvature_light(sub_syms, coords_mode)

    npz_name = f"{tag}_fe4s4_modeB.npz"
    np.savez(npz_name,
             symbols=sub_syms,
             coords_base=sub_coords,
             coords_mode=coords_mode,
             kappa_base=k_base,
             kappa_mode=k_mode,
             tag=tag)
    print("  Saved NPZ:", npz_name, "SHA512:", sha512_of_file(npz_name))

    rows = delta_teeth(k_base, k_mode)
    json_name = f"{tag}_DeltaTeeth.json"
    with open(json_name, "w") as f:
        json.dump(rows, f, indent=2)
    print("  Saved JSON:", json_name, "SHA512:", sha512_of_file(json_name))

Working in: /content/gqr_fe4s4_ispG

=== PATCH 4S3D_stageD from 4S3D.cif ===


RuntimeError: Not enough Fe/S atoms.

In [ ]:
!pip install gemmi pyscf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 MB 18.4 MB/s eta 0:00:00


In [ ]:
coords, syms = load_fes_from_cif("4S3D.cif")
print("4S3D Fe count:", np.sum(syms=="Fe"), " S count:", np.sum(syms=="S"))

coordsF, symsF = load_fes_from_cif("4S3F.cif")
print("4S3F Fe count:", np.sum(symsF=="Fe"), " S count:", np.sum(symsF=="S"))

4S3D Fe count: 3  S count: 19
4S3F Fe count: 3  S count: 20


In [ ]:
# ============================
# BLOCK M0 — INSTALL + IMPORT
# ============================

!pip install -q gemmi pyscf

import os, json, hashlib
import numpy as np
import gemmi
from pyscf import gto, dft
from pyscf.dft import numint

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 MB 18.5 MB/s eta 0:00:00


In [ ]:
# ============================
# BLOCK M1 — CREATE PROJECT DIR + MOVE CIF FILES
# ============================

project_root = "/content/gqr_fe4s4_ispG_DF_patch"
os.makedirs(project_root, exist_ok=True)

# Move ONLY D and F CIF files (do not overwrite good files)
!mv -n /content/4S3D.cif "$project_root"/ 2>/dev/null
!mv -n /content/4S3F.cif "$project_root"/ 2>/dev/null

# Verify contents
os.chdir(project_root)
print("Working in:", os.getcwd())
!ls -lh

Working in: /content/gqr_fe4s4_ispG_DF_patch
total 1.4M
-rw-r--r-- 1 root root 678K Nov 22 14:02 4S3D.cif
-rw-r--r-- 1 root root 685K Nov 22 14:02 4S3F.cif


In [ ]:
import os, shutil

main_root  = "/content/gqr_fe4s4_ispG"
patch_root = "/content/gqr_fe4s4_ispG_DF_patch"  # your patch folder

# make sure both exist
os.makedirs(main_root, exist_ok=True)
os.makedirs(patch_root, exist_ok=True)

for name in ["4S3D.cif", "4S3F.cif"]:
    src = os.path.join(patch_root, name)
    dst = os.path.join(main_root,  name)
    if os.path.exists(src):
        print(f"Copying {src} → {dst}")
        shutil.copy2(src, dst)
    else:
        print("Not found in patch folder:", src)

print("\nMain folder now contains:")
os.chdir(main_root)
!ls -lh

Copying /content/gqr_fe4s4_ispG_DF_patch/4S3D.cif → /content/gqr_fe4s4_ispG/4S3D.cif
Copying /content/gqr_fe4s4_ispG_DF_patch/4S3F.cif → /content/gqr_fe4s4_ispG/4S3F.cif

Main folder now contains:
total 1.4M
-rw-r--r-- 1 root root 678K Nov 22 14:02 4S3D.cif
-rw-r--r-- 1 root root 685K Nov 22 14:02 4S3F.cif


In [ ]:
import os, json, hashlib
import numpy as np
import gemmi
from pyscf import gto, dft
from pyscf.dft import numint

# ------------------------------
# Setup
# ------------------------------
project_root = "/content/gqr_fe4s4_ispG"
os.makedirs(project_root, exist_ok=True)
os.chdir(project_root)
print("Working in:", os.getcwd())

# ------------------------------
# Helpers
# ------------------------------
def sha512_of_file(path):
    h = hashlib.sha512()
    with open(path, "rb") as f:
        for chunk in iter(lambda: f.read(8192), b""):
            h.update(chunk)
    return h.hexdigest()

def load_fes_from_cif(cif_path):
    doc = gemmi.cif.read_file(cif_path)
    block = doc.sole_block()
    st    = gemmi.make_structure_from_block(block)
    model = st[0]

    coords, syms = [], []
    for ch in model:
        for res in ch:
            for at in res:
                el = at.element.name.upper()
                if el.startswith("FE") or el == "S":
                    syms.append("Fe" if el.startswith("FE") else "S")
                    coords.append([at.pos.x, at.pos.y, at.pos.z])
    return np.array(coords), np.array(syms)

def select_fe3s4_cluster(coords, syms, max_s=4):
    """
    Build an Fe3S4-like cluster:
      - use ALL Fe present (we know it's 3 here)
      - pick up to max_s nearest S atoms to the Fe centroid
    """
    fe_idx = np.where(syms == "Fe")[0]
    s_idx  = np.where(syms == "S")[0]

    if fe_idx.size == 0 or s_idx.size == 0:
        raise RuntimeError("No Fe or no S atoms in CIF.")

    # Use all Fe (for D/F this is 3)
    fe_sel = fe_idx
    fe_centroid = coords[fe_sel].mean(axis=0)

    # Sort S by distance to Fe centroid
    dists = np.linalg.norm(coords[s_idx] - fe_centroid, axis=1)
    order = np.argsort(dists)
    n_s_use = min(max_s, s_idx.size)
    s_sel = s_idx[order[:n_s_use]]

    sel = np.concatenate([fe_sel, s_sel])
    print(f"  Selected {fe_sel.size} Fe and {n_s_use} S (indices: {sel})")
    return sel

def curvature_light(symbols, coords, box=3.0, ngrid=24):
    atom_str = "\n".join(f"{s} {x} {y} {z}"
                         for s,(x,y,z) in zip(symbols, coords))
    mol = gto.M(atom=atom_str, basis="sto-3g", spin=0, charge=0)
    mf  = dft.RKS(mol); mf.xc="PBE"; mf.max_cycle=80; mf.conv_tol=1e-5
    mf.kernel()

    xs = np.linspace(-box, box, ngrid)
    X, Y, Z = np.meshgrid(xs, xs, xs, indexing="ij")
    grid  = np.stack([X.ravel(),Y.ravel(),Z.ravel()],axis=1)

    ni = numint.NumInt()
    ao = ni.eval_ao(mol, grid)
    rho = ni.eval_rho(mol, ao, mf.make_rdm1()).reshape((ngrid,ngrid,ngrid))

    dx = xs[1] - xs[0]
    lap = (np.gradient(np.gradient(rho,dx,axis=0),dx,axis=0) +
           np.gradient(np.gradient(rho,dx,axis=1),dx,axis=1) +
           np.gradient(np.gradient(rho,dx,axis=2),dx,axis=2))

    mask = rho > 1e-3
    kappa = np.abs(lap[mask].ravel())
    return np.clip(kappa, 0, np.percentile(kappa, 99.5))

def modeB(coords, symbols, amp=0.20):
    C = coords.copy()
    Fe = np.where(symbols == "Fe")[0]
    S  = np.where(symbols == "S")[0]

    pairs = []
    for fi in Fe:
        r = C[fi]
        d = [np.linalg.norm(C[si]-r) for si in S]
        si = S[int(np.argmin(d))]
        pairs.append((fi, si))

    def stretch(C, fi, si, d):
        v = C[si] - C[fi]
        L = np.linalg.norm(v)
        if L < 1e-6:
            return C
        u = v/L
        C[fi] -= 0.5*d*u
        C[si] += 0.5*d*u
        return C

    if len(pairs) >= 2:
        stretch(C, pairs[0][0], pairs[0][1], +amp)
        stretch(C, pairs[1][0], pairs[1][1], -amp)
    return C

L_EFF = 3.0
BINS  = list(range(40,126))

def delta_teeth(kappa_base, kappa_mode):
    rows = []
    for b in BINS:
        cb, _ = np.histogram(kappa_base, b, density=True)
        cm, _ = np.histogram(kappa_mode, b, density=True)
        tb = int(np.sum(cb < 1e-9))
        tm = int(np.sum(cm < 1e-9))
        rows.append({
            "bins": b,
            "Delta_r_A": L_EFF / b,
            "Delta_teeth": tm - tb
        })
    return rows

# ------------------------------
# PATCH D & F AS Fe3S4-LIKE CLUSTERS
# ------------------------------
for tag, cif in [("4S3D_stageD", "4S3D.cif"),
                 ("4S3F_stageF", "4S3F.cif")]:
    print(f"\n=== PATCH {tag} from {cif} ===")
    coords, syms = load_fes_from_cif(cif)
    sel = select_fe3s4_cluster(coords, syms, max_s=4)
    sub_syms   = syms[sel]
    sub_coords = coords[sel] - coords[sel].mean(axis=0)

    # base
    k_base = curvature_light(sub_syms, sub_coords)
    # mode-B distorted
    coords_mode = modeB(sub_coords, sub_syms, amp=0.20)
    k_mode = curvature_light(sub_syms, coords_mode)

    # NPZ
    npz_name = f"{tag}_Fe3S4_modeB.npz"
    np.savez(npz_name,
             symbols=sub_syms,
             coords_base=sub_coords,
             coords_mode=coords_mode,
             kappa_base=k_base,
             kappa_mode=k_mode,
             tag=tag)
    print("  Saved NPZ:", npz_name, "SHA512:", sha512_of_file(npz_name))

    # JSON
    rows = delta_teeth(k_base, k_mode)
    json_name = f"{tag}_Fe3S4_DeltaTeeth.json"
    with open(json_name, "w") as f:
        json.dump(rows, f, indent=2)
    print("  Saved JSON:", json_name, "SHA512:", sha512_of_file(json_name))

Working in: /content/gqr_fe4s4_ispG

=== PATCH 4S3D_stageD from 4S3D.cif ===
  Selected 3 Fe and 4 S (indices: [15 16 17 20 18 21 19])
SCF not converged.
SCF energy = -5293.80155660875
SCF not converged.
SCF energy = -5295.55733006249
  Saved NPZ: 4S3D_stageD_Fe3S4_modeB.npz SHA512: 96665ab8bb088feff923e8ee4d0a3b828bfb788f14f226eca4db93cb5bc043a8146a513c9cc62aee53b2bbec8e733b536434ed11091e911a1ee00db7bf6528bc
  Saved JSON: 4S3D_stageD_Fe3S4_DeltaTeeth.json SHA512: cf003a13d6273c3701cbf5ed2f538ce419d55fddeb33d5654d96161c61bea487550a7a7ce2d415b592ae599a4edbc32501ad728d3c8db14231bd112c30e18ac5

=== PATCH 4S3F_stageF from 4S3F.cif ===
  Selected 3 Fe and 4 S (indices: [16 17 18 21 22 20 19])
SCF not converged.
SCF energy = -5295.78523623509
SCF not converged.
SCF energy = -5293.76471485942
  Saved NPZ: 4S3F_stageF_Fe3S4_modeB.npz SHA512: 65ed81188ae91f1894c56084755c4ba5549be815a596569427e23a44b2fb5ded5424e737b7ffb175c73523d34646dc392186a2d5f8607ee906ee4d7fb5803f96
  Saved JSON: 4S3F_stageF

In [ ]:
import sys
import os
import urllib.request
import numpy as np
import gemmi
from pyscf import gto, dft, data

# ==========================================
# 1. CONFIGURATION
# ==========================================
BASIS_SET = 'def2-svp'   # Validation grade (Step up from STO-3G)
FUNCTIONAL = 'pbe'
GRID_POINTS = 32         # 32^3 = 32,768 points (Matches paper)
BOX_MARGIN = 3.5         # Angstroms buffer around cluster
DISPLACEMENT = 0.20      # Angstroms (Mode B amplitude)

class FeS_Analyzer:
    def __init__(self, cif_path):
        self.cif_path = cif_path
        self.structure = gemmi.read_structure(cif_path)
        self.model = self.structure[0]
        self.atoms = [] # List of (Element, x, y, z) tuples

    def extract_cluster(self, residue_name="SF4"):
        """
        Extracts the Fe4S4 cluster coordinates using Gemmi.
        Default looks for 'SF4' residue, but can be adapted for
        pure connectivity search if needed.
        """
        print(f"Loading {self.cif_path}...")
        cluster_atoms = []

        # Iterate to find the cluster
        for chain in self.model:
            for res in chain:
                if res.name == residue_name:
                    print(f"  Found {residue_name} in Chain {chain.name} Residue {res.seqid}")
                    for atom in res:
                        # Store as [Element, x, y, z]
                        pos = atom.pos
                        cluster_atoms.append({
                            'elem': atom.element.name,
                            'pos': np.array([pos.x, pos.y, pos.z])
                        })

        if not cluster_atoms:
            raise ValueError(f"No {residue_name} cluster found in {self.cif_path}")

        # Center the cluster at Origin (0,0,0) as per Methods
        positions = np.array([a['pos'] for a in cluster_atoms])
        centroid = np.mean(positions, axis=0)

        self.atoms = []
        for atom in cluster_atoms:
            centered_pos = atom['pos'] - centroid
            self.atoms.append((atom['elem'], centered_pos))

        print(f"  Extracted {len(self.atoms)} atoms and centered geometry.")

    def apply_mode_b_distortion(self):
        """
        Applies 'Mode B' Asymmetric Stretch.
        Logic from paper: "For each Fe, the closest S partner was identified
        and displaced along the Fe--S bond vector."
        """
        distorted_atoms = []

        # Separate Fe and S indices
        fe_indices = [i for i, a in enumerate(self.atoms) if a[0] == 'Fe']
        s_indices = [i for i, a in enumerate(self.atoms) if a[0] == 'S']

        current_positions = np.array([a[1] for a in self.atoms])

        # Create a copy for distortion
        new_positions = current_positions.copy()

        print("  Applying Mode B Distortion...")

        # Apply logic: Find nearest S for each Fe
        for fe_idx in fe_indices:
            fe_pos = current_positions[fe_idx]
            min_dist = np.inf
            nearest_s_idx = -1

            for s_idx in s_indices:
                dist = np.linalg.norm(fe_pos - current_positions[s_idx])
                if dist < min_dist:
                    min_dist = dist
                    nearest_s_idx = s_idx

            # Vector Calculation
            s_pos = current_positions[nearest_s_idx]
            bond_vector = s_pos - fe_pos
            unit_vector = bond_vector / np.linalg.norm(bond_vector)

            # Displace Fe along vector (+/- 0.20 A)
            # Paper mentions "opposite directions" for pairs.
            # We apply +0.20 to Fe towards S (compression/stretch)
            displacement_vector = unit_vector * DISPLACEMENT

            # Apply to Fe (and optionally S if breathing mode, but here just Fe stretch)
            new_positions[fe_idx] += displacement_vector

        # Reconstruct atom list
        for i, atom_data in enumerate(self.atoms):
            distorted_atoms.append((atom_data[0], new_positions[i]))

        return distorted_atoms

    def atom_list_to_pyscf_string(self, atom_list):
        """Converts internal list to PySCF string format: 'Fe 1.0 0.0 0.0; S ...'"""
        return "; ".join([f"{a[0]} {a[1][0]:.5f} {a[1][1]:.5f} {a[1][2]:.5f}" for a in atom_list])

# ==========================================
# 2. PySCF & ANALYSIS ENGINE
# ==========================================

def get_curvature_traps(atom_string, name):
    """Runs SCF and counts Curvature Traps"""
    mol = gto.M(atom=atom_string, basis=BASIS_SET, charge=-2, spin=0, verbose=0, unit='Angstrom')
    mol.build()

    print(f"  Running DFT ({FUNCTIONAL}/{BASIS_SET}) for {name}...")
    mf = dft.RKS(mol)
    mf.xc = FUNCTIONAL
    mf.run()

    if not mf.converged:
        print("  WARNING: SCF not converged!")

    # Grid Setup for Laplacian
    print("  Calculating Laplacian on 32k grid...")
    coords = mol.atom_coords()
    min_c = coords.min(axis=0) - BOX_MARGIN
    max_c = coords.max(axis=0) + BOX_MARGIN

    xs = np.linspace(min_c[0], max_c[0], GRID_POINTS)
    ys = np.linspace(min_c[1], max_c[1], GRID_POINTS)
    zs = np.linspace(min_c[2], max_c[2], GRID_POINTS)

    # 3. Calculate Laplacian
    # Note: For speed in this script, we calculate rho then numerical laplacian.
    # Ideally, analytical Hessian is better but expensive.

    mx, my, mz = np.meshgrid(xs, ys, zs, indexing='ij')
    grid_coords = np.vstack([mx.ravel(), my.ravel(), mz.ravel()]).T

    ao = mol.eval_gto("GTOval_sph", grid_coords)
    rho = dft.numint.eval_rho(mol, ao, mf.make_rdm1(), xctype='LDA')
    rho_cube = rho.reshape(GRID_POINTS, GRID_POINTS, GRID_POINTS)

    # Finite Difference Laplacian
    dx = xs[1] - xs[0]
    lap = (np.gradient(np.gradient(rho_cube, axis=0), axis=0)/dx**2 +
           np.gradient(np.gradient(rho_cube, axis=1), axis=1)/dx**2 +
           np.gradient(np.gradient(rho_cube, axis=2), axis=2)/dx**2)

    abs_lap = np.abs(lap.ravel())

    # Count Traps (PDF < 1e-9)
    # Using histogram binning as per paper
    hist, _ = np.histogram(abs_lap[abs_lap < np.percentile(abs_lap, 99.5)], bins=200, density=True)
    n_traps = np.sum(hist < 1e-9)

    return n_traps

# ==========================================
# 3. MAIN RUNNER
# ==========================================

def run_pipeline(active_cif, inhibited_cif):
    print("=== STARTING CURVATURE TRAP PIPELINE ===")

    # 1. Active Cluster (4S39)
    active = FeS_Analyzer(active_cif)
    active.extract_cluster()

    active_base_str = active.atom_list_to_pyscf_string(active.atoms)
    active_dist_str = active.atom_list_to_pyscf_string(active.apply_mode_b_distortion())

    traps_active_base = get_curvature_traps(active_base_str, "Active Base")
    traps_active_dist = get_curvature_traps(active_dist_str, "Active Distorted")

    # 2. Inhibited Cluster (4S3E)
    inhib = FeS_Analyzer(inhibited_cif)
    inhib.extract_cluster()

    inhib_base_str = inhib.atom_list_to_pyscf_string(inhib.atoms)
    inhib_dist_str = inhib.atom_list_to_pyscf_string(inhib.apply_mode_b_distortion())

    traps_inhib_base = get_curvature_traps(inhib_base_str, "Inhibited Base")
    traps_inhib_dist = get_curvature_traps(inhib_dist_str, "Inhibited Distorted")

    # 3. Calculate Differentials
    delta_active = traps_active_dist - traps_active_base
    delta_inhib = traps_inhib_dist - traps_inhib_base

    print("\n" + "="*40)
    print("RESULTS SUMMARY")
    print("="*40)
    print(f"Active (4S39) Delta Teeth:    {delta_active}")
    print(f"Inhibited (4S3E) Delta Teeth: {delta_inhib}")
    print("="*40)

    if delta_active > 0 and delta_inhib <= 0:
        print(">> HYPOTHESIS CONFIRMED: Inhibitor collapses trap resonance.")
    else:
        print(">> RESULT AMBIGUOUS: Check basis set or grid density.")

if __name__ == "__main__":
    # Defined paths
    p_active = "/content/4s39.cif"
    p_inhib  = "/content/4s3e.cif"

    # Auto-download if missing
    for p, pdb_id in [(p_active, "4S39"), (p_inhib, "4S3E")]:
        if not os.path.exists(p):
            url = f"https://files.rcsb.org/download/{pdb_id}.cif"
            print(f"Downloading {pdb_id} from {url}...")
            try:
                urllib.request.urlretrieve(url, p)
            except Exception as e:
                print(f"Failed to download {pdb_id}: {e}")

    # Run with absolute paths
    run_pipeline(p_active, p_inhib)

    # ... (Keep the FeS_Analyzer class and imports from the previous script) ...

def analyze_and_save(atom_string, name, file_tag):
    """
    Runs SCF, calculates Laplacian, counts traps, and SAVES an .npz file.
    file_tag: distinct string for filename (e.g., 'active_base')
    """
    print(f"  [Processing] {name}...")

    # 1. Build & Run DFT
    mol = gto.M(atom=atom_string, basis=BASIS_SET, charge=-2, spin=0, verbose=0, unit='Angstrom')
    mol.build()

    mf = dft.RKS(mol)
    mf.xc = FUNCTIONAL
    mf.run()

    if not mf.converged:
        print(f"  [WARNING] SCF did not converge for {name}")

    # 2. Grid & Laplacian
    coords = mol.atom_coords()
    min_c = coords.min(axis=0) - BOX_MARGIN
    max_c = coords.max(axis=0) + BOX_MARGIN

    xs = np.linspace(min_c[0], max_c[0], GRID_POINTS)
    ys = np.linspace(min_c[1], max_c[1], GRID_POINTS)
    zs = np.linspace(min_c[2], max_c[2], GRID_POINTS)

    mx, my, mz = np.meshgrid(xs, ys, zs, indexing='ij')
    grid_coords = np.vstack([mx.ravel(), my.ravel(), mz.ravel()]).T

    # Calculate Density & Laplacian
    ao = mol.eval_gto("GTOval_sph", grid_coords)
    rho = dft.numint.eval_rho(mol, ao, mf.make_rdm1(), xctype='LDA')
    rho_cube = rho.reshape(GRID_POINTS, GRID_POINTS, GRID_POINTS)

    dx = xs[1] - xs[0]
    lap = (np.gradient(np.gradient(rho_cube, axis=0), axis=0)/dx**2 +
           np.gradient(np.gradient(rho_cube, axis=1), axis=1)/dx**2 +
           np.gradient(np.gradient(rho_cube, axis=2), axis=2)/dx**2)

    abs_lap = np.abs(lap.ravel())

    # 3. Filter & Histogram (The "Comb")
    # Cap at 99.5th percentile to remove core-electron spikes
    cap = np.percentile(abs_lap, 99.5)
    filtered_lap = abs_lap[abs_lap < cap]

    hist_counts, bin_edges = np.histogram(filtered_lap, bins=200, density=True)

    # Count Traps (where PDF is essentially zero)
    n_traps = np.sum(hist_counts < 1e-9)

    # 4. SAVE TO DISK
    outfile = f"{file_tag}_{BASIS_SET}.npz"
    print(f"  [Saving] -> {outfile}")

    np.savez_compressed(
        outfile,
        laplacian_raw=abs_lap,     # The full 3D grid (flattened)
        hist_density=hist_counts,  # The Y-axis of your "Comb"
        hist_bins=bin_edges,       # The X-axis of your "Comb"
        trap_count=n_traps,        # The scalar result
        atoms=atom_string,         # The geometry used
        grid_info=np.array([GRID_POINTS, BOX_MARGIN])
    )

    return n_traps

def run_pipeline(active_cif, inhibited_cif):
    print(f"=== STARTING RUN ({BASIS_SET}) ===")

    # 1. Setup Structures
    active = FeS_Analyzer(active_cif)
    active.extract_cluster()
    inhib = FeS_Analyzer(inhibited_cif)
    inhib.extract_cluster()

    # 2. Run Active
    act_base_s = active.atom_list_to_pyscf_string(active.atoms)
    act_dist_s = active.atom_list_to_pyscf_string(active.apply_mode_b_distortion())

    t_act_base = analyze_and_save(act_base_s, "Active Base", "4S39_active_base")
    t_act_dist = analyze_and_save(act_dist_s, "Active Dist", "4S39_active_dist")

    # 3. Run Inhibited
    inh_base_s = inhib.atom_list_to_pyscf_string(inhib.atoms)
    inh_dist_s = inhib.atom_list_to_pyscf_string(inhib.apply_mode_b_distortion())

    t_inh_base = analyze_and_save(inh_base_s, "Inhibited Base", "4S3E_inhib_base")
    t_inh_dist = analyze_and_save(inh_dist_s, "Inhibited Dist", "4S3E_inhib_dist")

    # 4. Summary
    print("\n=== SUMMARY ===")
    print(f"Active Delta:    {t_act_dist - t_act_base}")
    print(f"Inhibited Delta: {t_inh_dist - t_inh_base}")



=== STARTING CURVATURE TRAP PIPELINE ===
Loading /content/4s39.cif...
  Found SF4 in Chain A Residue 901
  Extracted 8 atoms and centered geometry.
  Applying Mode B Distortion...
  Running DFT (pbe/def2-svp) for Active Base...


KeyboardInterrupt: 

In [ ]:
import sys
import numpy as np
import gemmi
from pyscf import gto, dft, data

# ==========================================
# 1. CONFIGURATION
# ==========================================
# CHANGE THIS TO 'def2-svp' or 'def2-tzvp' FOR THE PAPER
BASIS_SET = 'def2-svp'
FUNCTIONAL = 'pbe'
GRID_POINTS = 32         # 32^3 grid
BOX_MARGIN = 3.5         # Angstrom buffer
DISPLACEMENT = 0.20      # Mode B Amplitude

# ==========================================
# 2. CLASS: STRUCTURE EXTRACTOR
# ==========================================
class FeS_Analyzer:
    def __init__(self, cif_path):
        self.cif_path = cif_path
        try:
            self.structure = gemmi.read_structure(cif_path)
        except Exception as e:
            print(f"Error reading CIF: {e}")
            sys.exit(1)
        self.model = self.structure[0]
        self.atoms = []

    def extract_cluster(self, residue_name="SF4"):
        print(f"  [Loader] Reading {self.cif_path}...")
        cluster_atoms = []

        # Search for residue
        for chain in self.model:
            for res in chain:
                if residue_name in res.name:
                    for atom in res:
                        pos = atom.pos
                        cluster_atoms.append({
                            'elem': atom.element.name,
                            'pos': np.array([pos.x, pos.y, pos.z])
                        })

        if not cluster_atoms:
            # Fallback for 4S3D/F (Fe3S4) if labeled differently
            print(f"    Note: Exact '{residue_name}' not found. Searching for Fe/S clusters manually...")
            # (Simple fallback logic could go here, but usually SF4 or F3S works)
            raise ValueError(f"Cluster extraction failed for {self.cif_path}")

        # Center at Origin
        positions = np.array([a['pos'] for a in cluster_atoms])
        centroid = np.mean(positions, axis=0)

        self.atoms = []
        for atom in cluster_atoms:
            centered_pos = atom['pos'] - centroid
            self.atoms.append((atom['elem'], centered_pos))

        print(f"    Extracted {len(self.atoms)} atoms.")

    def apply_mode_b_distortion(self):
        """Applies Asymmetric Fe-S Stretch (Mode B)"""
        distorted_atoms = []
        fe_indices = [i for i, a in enumerate(self.atoms) if a[0] == 'Fe']
        s_indices = [i for i, a in enumerate(self.atoms) if a[0] == 'S']

        current_positions = np.array([a[1] for a in self.atoms])
        new_positions = current_positions.copy()

        # Vector displacement logic
        for fe_idx in fe_indices:
            fe_pos = current_positions[fe_idx]
            min_dist = np.inf
            nearest_s_idx = -1

            for s_idx in s_indices:
                dist = np.linalg.norm(fe_pos - current_positions[s_idx])
                if dist < min_dist:
                    min_dist = dist
                    nearest_s_idx = s_idx

            s_pos = current_positions[nearest_s_idx]
            bond_vector = s_pos - fe_pos
            unit_vector = bond_vector / np.linalg.norm(bond_vector)

            # Apply displacement (+0.20 A along bond)
            new_positions[fe_idx] += unit_vector * DISPLACEMENT

        for i, atom_data in enumerate(self.atoms):
            distorted_atoms.append((atom_data[0], new_positions[i]))

        return distorted_atoms

    def atom_list_to_pyscf_string(self, atom_list):
        return "; ".join([f"{a[0]} {a[1][0]:.5f} {a[1][1]:.5f} {a[1][2]:.5f}" for a in atom_list])

# ==========================================
# 3. FUNCTION: ANALYSIS & SAVING
# ==========================================
def analyze_and_save(atom_string, name, file_tag):
    """Runs PySCF, calculates Laplacian, and saves .npz"""
    print(f"  [Calc] Running DFT ({BASIS_SET}) for {name}...")

    # Build Molecule
    mol = gto.M(atom=atom_string, basis=BASIS_SET, charge=-2, spin=0, verbose=0, unit='Angstrom')
    mol.build()

    # Run DFT
    mf = dft.RKS(mol)
    mf.xc = FUNCTIONAL
    mf.run()

    if not mf.converged:
        print(f"    WARNING: SCF Not Converged for {name}")

    # Grid Generation
    coords = mol.atom_coords()
    min_c = coords.min(axis=0) - BOX_MARGIN
    max_c = coords.max(axis=0) + BOX_MARGIN

    xs = np.linspace(min_c[0], max_c[0], GRID_POINTS)
    ys = np.linspace(min_c[1], max_c[1], GRID_POINTS)
    zs = np.linspace(min_c[2], max_c[2], GRID_POINTS)

    mx, my, mz = np.meshgrid(xs, ys, zs, indexing='ij')

    # Calculate Laplacian (Numerical)
    # 1. Density
    ao = mol.eval_gto("GTOval_sph", np.vstack([mx.ravel(), my.ravel(), mz.ravel()]).T)
    rho = dft.numint.eval_rho(mol, ao, mf.make_rdm1(), xctype='LDA')
    rho_cube = rho.reshape(GRID_POINTS, GRID_POINTS, GRID_POINTS)

    # 2. Laplacian
    dx = xs[1] - xs[0]
    lap = (np.gradient(np.gradient(rho_cube, axis=0), axis=0)/dx**2 +
           np.gradient(np.gradient(rho_cube, axis=1), axis=1)/dx**2 +
           np.gradient(np.gradient(rho_cube, axis=2), axis=2)/dx**2)

    abs_lap = np.abs(lap.ravel())

    # 3. Trap Counting
    cap = np.percentile(abs_lap, 99.5)
    clean_lap = abs_lap[abs_lap < cap]

    hist, bin_edges = np.histogram(clean_lap, bins=200, density=True)
    n_traps = np.sum(hist < 1e-9)

    # 4. Save Data
    outfile = f"{file_tag}.npz"
    np.savez_compressed(outfile,
                        laplacian=abs_lap,
                        hist_density=hist,
                        hist_bins=bin_edges,
                        trap_count=n_traps)
    print(f"    Saved -> {outfile} (Traps: {n_traps})")

    return n_traps

# ==========================================
# 4. ORCHESTRATOR
# ==========================================
def run_pipeline(active_path, inhib_path):
    print("=== INITIALIZING PIPELINE ===")

    # 1. Extract Geometries
    active_analyzer = FeS_Analyzer(active_path)
    active_analyzer.extract_cluster()

    inhib_analyzer = FeS_Analyzer(inhib_path)
    inhib_analyzer.extract_cluster()

    # 2. Prepare PySCF Strings
    act_base = active_analyzer.atom_list_to_pyscf_string(active_analyzer.atoms)
    act_dist = active_analyzer.atom_list_to_pyscf_string(active_analyzer.apply_mode_b_distortion())

    inh_base = inhib_analyzer.atom_list_to_pyscf_string(inhib_analyzer.atoms)
    inh_dist = inhib_analyzer.atom_list_to_pyscf_string(inhib_analyzer.apply_mode_b_distortion())

    # 3. Execute Quantum Calcs
    print("\n--- Processing Active Cluster ---")
    t1 = analyze_and_save(act_base, "Active Base", "active_base")
    t2 = analyze_and_save(act_dist, "Active Distorted", "active_dist")

    print("\n--- Processing Inhibited Cluster ---")
    t3 = analyze_and_save(inh_base, "Inhibited Base", "inhib_base")
    t4 = analyze_and_save(inh_dist, "Inhibited Distorted", "inhib_dist")

    # 4. Final Report
    delta_act = t2 - t1
    delta_inh = t4 - t3

    print("\n" + "="*40)
    print(f"Basis Set: {BASIS_SET}")
    print(f"Active Delta (Teeth):    {delta_act}")
    print(f"Inhibited Delta (Teeth): {delta_inh}")
    print("="*40)


# ==========================================
# 5. EXECUTION (Auto-Download Version)
# ==========================================
import os
import urllib.request

def fetch_pdb_if_missing(pdb_code):
    """Downloads CIF from RCSB if not found locally"""
    filename = f"{pdb_code.lower()}.cif"

    # Check if file exists in current directory
    if os.path.exists(filename):
        print(f"  [Found] {filename}")
        return filename

    # If not, download it
    url = f"https://files.rcsb.org/download/{pdb_code}.cif"
    print(f"  [Downloading] {pdb_code} from RCSB...")
    try:
        urllib.request.urlretrieve(url, filename)
        print(f"  [Saved] -> {filename}")
    except Exception as e:
        print(f"  [Error] Could not download {pdb_code}: {e}")
        raise

    return filename

if __name__ == "__main__":

    # 1. Auto-download the structures needed for the paper
    #    Active = 4S39, Inhibited = 4S3E
    try:
        p_active = fetch_pdb_if_missing("4S39")
        p_inhib  = fetch_pdb_if_missing("4S3E")

        # 2. Run the pipeline on the local files
        #    Note: os.path.abspath ensures Gemmi finds it regardless of folder depth
        run_pipeline(os.path.abspath(p_active), os.path.abspath(p_inhib))

    except Exception as e:
        print(f"\nCRITICAL FAILURE: {e}")


  [Found] 4s39.cif
  [Found] 4s3e.cif
=== INITIALIZING PIPELINE ===
  [Loader] Reading /content/4s39.cif...
    Extracted 8 atoms.
  [Loader] Reading /content/4s3e.cif...
    Extracted 8 atoms.

--- Processing Active Cluster ---
  [Calc] Running DFT (def2-svp) for Active Base...


KeyboardInterrupt: 

In [ ]:
import sys
import os
import urllib.request
import numpy as np
import gemmi
from pyscf import gto, dft, data

# ==========================================
# 1. CONFIGURATION
# ==========================================
BASIS_SET = 'def2-svp'
FUNCTIONAL = 'pbe'
GRID_POINTS = 32
BOX_MARGIN = 3.5
DISPLACEMENT = 0.20

# ==========================================
# 2. CLASS: STRUCTURE EXTRACTOR
# ==========================================
class FeS_Analyzer:
    def __init__(self, cif_path):
        self.cif_path = cif_path
        try:
            self.structure = gemmi.read_structure(cif_path)
        except Exception as e:
            print(f"Error reading CIF: {e}")
            sys.exit(1)
        self.model = self.structure[0]
        self.atoms = []

    def extract_cluster(self, residue_name="SF4"):
        print(f"  [Loader] Reading {self.cif_path}...")
        cluster_atoms = []
        for chain in self.model:
            for res in chain:
                if residue_name in res.name:
                    for atom in res:
                        pos = atom.pos
                        cluster_atoms.append({
                            'elem': atom.element.name,
                            'pos': np.array([pos.x, pos.y, pos.z])
                        })
        if not cluster_atoms:
            raise ValueError(f"Cluster extraction failed for {self.cif_path}")

        positions = np.array([a['pos'] for a in cluster_atoms])
        centroid = np.mean(positions, axis=0)

        self.atoms = []
        for atom in cluster_atoms:
            self.atoms.append((atom['elem'], atom['pos'] - centroid))
        print(f"    Extracted {len(self.atoms)} atoms.")

    def apply_mode_b_distortion(self):
        distorted_atoms = []
        fe_indices = [i for i, a in enumerate(self.atoms) if a[0] == 'Fe']
        s_indices = [i for i, a in enumerate(self.atoms) if a[0] == 'S']

        current_positions = np.array([a[1] for a in self.atoms])
        new_positions = current_positions.copy()

        for fe_idx in fe_indices:
            fe_pos = current_positions[fe_idx]
            min_dist = np.inf
            nearest_s_idx = -1
            for s_idx in s_indices:
                dist = np.linalg.norm(fe_pos - current_positions[s_idx])
                if dist < min_dist:
                    min_dist = dist
                    nearest_s_idx = s_idx

            s_pos = current_positions[nearest_s_idx]
            unit_vector = (s_pos - fe_pos) / np.linalg.norm(s_pos - fe_pos)
            new_positions[fe_idx] += unit_vector * DISPLACEMENT

        for i, atom_data in enumerate(self.atoms):
            distorted_atoms.append((atom_data[0], new_positions[i]))
        return distorted_atoms

    def atom_list_to_pyscf_string(self, atom_list):
        return "; ".join([f"{a[0]} {a[1][0]:.5f} {a[1][1]:.5f} {a[1][2]:.5f}" for a in atom_list])

# ==========================================
# 3. FUNCTION: ROBUST ANALYSIS (Newton-Raphson)
# ==========================================
def analyze_and_save(atom_string, name, file_tag):
    print(f"  [Calc] Running DFT ({BASIS_SET}) for {name}...")

    # 1. Build Molecule
    mol = gto.M(atom=atom_string, basis=BASIS_SET, charge=-2, spin=0, verbose=0, unit='Angstrom')
    mol.build()

    # 2. Try Standard DIIS Solver First
    mf = dft.RKS(mol)
    mf.xc = FUNCTIONAL
    mf.max_cycle = 50
    mf.run()

    # 3. Fallback: Newton-Raphson
    if not mf.converged:
        print(f"    [!] DIIS failed. Switching to Newton-Raphson solver...")
        mf = mf.newton()
        mf.run()

    if not mf.converged:
        print(f"    [CRITICAL FAILURE] SCF did not converge for {name}")
    else:
        print(f"    [Success] SCF Converged (E={mf.e_tot:.4f} Ha)")

    # 4. Grid & Laplacian
    coords = mol.atom_coords()
    min_c = coords.min(axis=0) - BOX_MARGIN
    max_c = coords.max(axis=0) + BOX_MARGIN

    xs = np.linspace(min_c[0], max_c[0], GRID_POINTS)
    ys = np.linspace(min_c[1], max_c[1], GRID_POINTS)
    zs = np.linspace(min_c[2], max_c[2], GRID_POINTS)
    mx, my, mz = np.meshgrid(xs, ys, zs, indexing='ij')

    ao = mol.eval_gto("GTOval_sph", np.vstack([mx.ravel(), my.ravel(), mz.ravel()]).T)
    rho = dft.numint.eval_rho(mol, ao, mf.make_rdm1(), xctype='LDA')
    rho_cube = rho.reshape(GRID_POINTS, GRID_POINTS, GRID_POINTS)

    dx = xs[1] - xs[0]
    lap = (np.gradient(np.gradient(rho_cube, axis=0), axis=0)/dx**2 +
           np.gradient(np.gradient(rho_cube, axis=1), axis=1)/dx**2 +
           np.gradient(np.gradient(rho_cube, axis=2), axis=2)/dx**2)

    abs_lap = np.abs(lap.ravel())

    # 5. Trap Counting & Saving
    cap = np.percentile(abs_lap, 99.5)
    clean_lap = abs_lap[abs_lap < cap]
    hist, bin_edges = np.histogram(clean_lap, bins=200, density=True)
    n_traps = np.sum(hist < 1e-9)

    outfile = f"{file_tag}.npz"
    np.savez_compressed(outfile, laplacian=abs_lap, hist_density=hist, hist_bins=bin_edges, trap_count=n_traps)
    print(f"    Saved -> {outfile} (Traps: {n_traps})")

    return n_traps

# ==========================================
# 4. ORCHESTRATOR
# ==========================================
def run_pipeline(active_path, inhib_path):
    print(f"=== STARTING RUN ({BASIS_SET}) ===")

    act_ana = FeS_Analyzer(active_path)
    act_ana.extract_cluster()
    inh_ana = FeS_Analyzer(inhib_path)
    inh_ana.extract_cluster()

    act_base = act_ana.atom_list_to_pyscf_string(act_ana.atoms)
    act_dist = act_ana.atom_list_to_pyscf_string(act_ana.apply_mode_b_distortion())
    inh_base = inh_ana.atom_list_to_pyscf_string(inh_ana.atoms)
    inh_dist = inh_ana.atom_list_to_pyscf_string(inh_ana.apply_mode_b_distortion())

    print("\n--- Processing Active Cluster ---")
    t1 = analyze_and_save(act_base, "Active Base", "active_base")
    t2 = analyze_and_save(act_dist, "Active Distorted", "active_dist")

    print("\n--- Processing Inhibited Cluster ---")
    t3 = analyze_and_save(inh_base, "Inhibited Base", "inhib_base")
    t4 = analyze_and_save(inh_dist, "Inhibited Distorted", "inhib_dist")

    delta_act = t2 - t1
    delta_inh = t4 - t3

    print("\n" + "="*40)
    print(f"Basis Set: {BASIS_SET}")
    print(f"Active Delta:    {delta_act}")
    print(f"Inhibited Delta: {delta_inh}")
    print("="*40)

# ==========================================
# 5. EXECUTION
# ==========================================
def fetch_pdb_if_missing(pdb_code):
    filename = f"{pdb_code.lower()}.cif"
    if os.path.exists(filename): return filename
    url = f"https://files.rcsb.org/download/{pdb_code}.cif"
    print(f"  [Downloading] {pdb_code}...")
    urllib.request.urlretrieve(url, filename)
    return filename

if __name__ == "__main__":
    try:
        p_active = fetch_pdb_if_missing("4S39")
        p_inhib  = fetch_pdb_if_missing("4S3E")
        run_pipeline(os.path.abspath(p_active), os.path.abspath(p_inhib))
    except Exception as e:
        print(f"\nCRITICAL FAILURE: {e}")


  [Downloading] 4S39...
  [Downloading] 4S3E...
=== STARTING RUN (def2-svp) ===
  [Loader] Reading /content/4s39.cif...
    Extracted 8 atoms.
  [Loader] Reading /content/4s3e.cif...
    Extracted 8 atoms.

--- Processing Active Cluster ---
  [Calc] Running DFT (def2-svp) for Active Base...
    [!] DIIS failed. Switching to Newton-Raphson solver...
    [Success] SCF Converged (E=-6645.0382 Ha)
    Saved -> active_base.npz (Traps: 55)
  [Calc] Running DFT (def2-svp) for Active Distorted...
    [!] DIIS failed. Switching to Newton-Raphson solver...
    [Success] SCF Converged (E=-6645.0510 Ha)
    Saved -> active_dist.npz (Traps: 42)

--- Processing Inhibited Cluster ---
  [Calc] Running DFT (def2-svp) for Inhibited Base...
    [!] DIIS failed. Switching to Newton-Raphson solver...
    [Success] SCF Converged (E=-6645.0287 Ha)
    Saved -> inhib_base.npz (Traps: 50)
  [Calc] Running DFT (def2-svp) for Inhibited Distorted...
    [!] DIIS failed. Switching to Newton-Raphson solver...
    [

In [ ]:
!pip install gemmi pyscf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 MB 17.2 MB/s eta 0:00:00
